<a href="https://colab.research.google.com/github/YShu7/FYP/blob/master/exp/final/LSTM_MultiClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2
import sqlite3
import operator
import pandas as pd
import collections
import networkx as nx
import json
import math
import numpy as np
from multiprocessing import Pool
from scipy.optimize import linear_sum_assignment
import time
import utils

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import numpy as np


# utils = __import__('utils.utils', fromlist=['object'])
# train = __import__('utils.train', fromlist=['object'])


from google.colab import drive
drive.mount('/content/drive')

torch.manual_seed(1)
torch.set_deterministic(True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/torch/__init__.py:422: UserWarning: torch.set_deterministic is deprecated and will be removed in a future release. Please use torch.use_deterministic_algorithms instead
  "torch.set_deterministic is deprecated and will be removed in a future "


In [ ]:
INTERVAL = 60
TIME_PERIOD = 1
SCALE_METERS = 15000
scaleMeters = SCALE_METERS * 0.2
SIGMA_V = scaleMeters / 100
IMPOSSIBLE = 1e-10
MINUTE = 10
UPDATE_TIME = MINUTE * INTERVAL

N, M = 30, 91
PATH = '/content/drive/MyDrive/FYP'
TRAIN_DB = 'database_100+_train_2.db'
TEST_DB = 'database_100-_test_2.db'

EMBEDDING_DIM = 32
HIDDEN_DIM = 128
INPUT_DIM = 10

# Dataset

In [ ]:
N, M = 30, 91
cnx = sqlite3.connect(f'{PATH}/{TRAIN_DB}')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)
df_contacts_tmp = pd.read_sql_query("SELECT * FROM contacts JOIN walkers ON walkers.id = contacts.walker_id", cnx)
idx = df_contacts_tmp.groupby(['walker_id', 'time'])['distance'].transform(min) == df_contacts_tmp['distance']
df_contacts = df_contacts_tmp[idx]

agent_to_ix = {}
for _, row in df_agents.iterrows():
  x, y = row['id'].split('EVEN')
  agent_to_ix[row['id']] = [int(x)/N, (int(y)%M)/M]

In [ ]:
class PathDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, inputs, targets, output_size, transform=None):
        self.inputs = inputs
        self.targets = targets
        self.output_size = output_size
        self.transform = transform

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        inputs = torch.tensor(self.inputs[idx]).to(device)
        targets = torch.tensor(self.targets[idx], dtype=torch.long).to(device)
        # targets = torch.tensor([1 if i == self.targets[idx] else 0 for i in range(self.output_size)], dtype=torch.long).to(device)
        # sample = (torch.reshape(inputs, (inputs.shape[0], 1, inputs.shape[1])), 
        #           torch.reshape(targets, (inputs.shape[0], 1)))
        sample = (inputs, targets)
        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
start_time_to_idx = collections.defaultdict(set)
for _, row in df_contacts.iterrows():
  start_time_to_idx[row['time']].add(row['agent_id'])
print(start_time_to_idx)

defaultdict(<class 'set'>, {1615273: {'2EVEN237', '4EVEN415', '7EVEN693', '4EVEN405', '6EVEN605', '8EVEN763', '8EVEN784', '15EVEN1434'}, 1615333: {'7EVEN674', '2EVEN237', '4EVEN415', '7EVEN693', '6EVEN606', '8EVEN785', '8EVEN784', '14EVEN1342'}, 1615513: {'2EVEN237', '4EVEN415', '7EVEN693', '10EVEN976', '6EVEN602', '8EVEN784', '9EVEN875', '5EVEN493', '3EVEN323', '5EVEN514', '8EVEN786'}, 1615573: {'2EVEN237', '4EVEN415', '7EVEN693', '5EVEN494', '6EVEN602', '3EVEN323', '5EVEN513', '8EVEN784', '8EVEN786', '10EVEN975'}, 1615633: {'9EVEN883', '2EVEN237', '7EVEN693', '4EVEN413', '5EVEN494', '6EVEN602', '5EVEN513', '8EVEN784', '8EVEN786', '8EVEN788'}, 1615693: {'2EVEN237', '5EVEN512', '5EVEN495', '4EVEN413', '6EVEN602', '5EVEN507', '8EVEN790', '8EVEN789', '8EVEN784', '8EVEN786'}, 1615753: {'2EVEN237', '5EVEN512', '5EVEN495', '5EVEN508', '4EVEN413', '6EVEN602', '7EVEN695', '8EVEN790', '8EVEN784'}, 1615813: {'2EVEN237', '5EVEN512', '5EVEN508', '7EVEN696', '6EVEN586', '6EVEN602', '7EVEN698', '8E

In [ ]:
def prepare_data(df_contacts, to_ix):
  real_ids = set(df_contacts['real_id'].tolist())
  input, candidate, target = [], [], []
  # For each words-list (sentence) and tags-list in each tuple of training_data

  for real_id in real_ids:
    walk = df_contacts.loc[df_contacts['real_id'] == real_id].sort_values('time')
    path, times = [], []

    for i, row in walk.iterrows():
      times.append(int(row['time']))
      path.append(row['agent_id'])

    start_idx = 0
    while True:
      try:
        curr_input = [to_ix[p] for p in path[start_idx:start_idx+10]]
        curr_target = int(path[start_idx+10].split('EVEN')[1])
        start_idx += INPUT_DIM

        input.append(curr_input)
        target.append(curr_target)
      except:
        break

  print(len(input), len(target))

  return input, target

In [ ]:
input_train, target_train = prepare_data(df_contacts, agent_to_ix)
train_loader = torch.utils.data.DataLoader(PathDataset(input_train, target_train, len(agent_to_ix)),
                                           batch_size=64, shuffle =True)

8161 8161


In [ ]:
cnx = sqlite3.connect(f'{PATH}/{TEST_DB}')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)
df_contacts_tmp = pd.read_sql_query("SELECT * FROM contacts JOIN walkers ON walkers.id = contacts.walker_id", cnx)
idx = df_contacts_tmp.groupby(['walker_id', 'time'])['distance'].transform(min) == df_contacts_tmp['distance']
df_contacts = df_contacts_tmp[idx]

input_val, target_val = prepare_data(df_contacts[:2000], agent_to_ix)
val_loader = torch.utils.data.DataLoader(PathDataset(input_val, target_val, len(agent_to_ix)),
                                         batch_size=1, shuffle =False)

input_test, target_test = prepare_data(df_contacts[2000:], agent_to_ix)
test_loader = torch.utils.data.DataLoader(PathDataset(input_test, target_test, len(agent_to_ix)), 
                                          batch_size=1, shuffle =False)

193 193
1653 1653


# Model

In [ ]:
DEBUG = False
def evaluate(model, loader, DEBUG=False):
  model.train(False)

  running_loss = 0
  running_corrects = 0

  for data in loader:

    inputs, targets = data
    outputs = model(inputs.type(torch.FloatTensor).to(device)).to(device)#.reshape(-1) #forward pass
    _, preds = torch.max(outputs, 1)
    # _, target_preds = torch.max(targets, 1)
    
    loss = criterion(outputs, targets.to(device))
    # loss = criterion(outputs, targets.type(torch.FloatTensor).to(device)) #obtain the loss function

    running_loss += loss.item()
    # running_corrects += torch.sum(preds == target_preds).item()
    running_corrects += torch.sum(preds == targets).item()

  epoch_loss = running_loss / len(loader)
  epochs_acc = running_corrects / len(loader)

  # if DEBUG: print(targets, preds)

  return epoch_loss, epochs_acc

#### Single

In [ ]:
# DEBUG = False

# class LSTMTagger(nn.Module):

#   def __init__(self, input_dim, embedding_dim, hidden_dim, num_classes, num_layers=1, seq_len=10):
#     super(LSTMTagger, self).__init__()
#     self.hidden_dim = hidden_dim
#     self.num_layers = num_layers

#     # CNN
#     # self.con3_layer = nn.Conv1d(in_channels=embedding_dim, out_channels=hidden_dim, kernel_size=3)
#     # self.con5_layer = nn.Conv1d(in_channels=embedding_dim, out_channels=hidden_dim, kernel_size=5)
#     # self.max_pool = nn.MaxPool1d(3, stride=2)

#     # LSTM
#     self.lstm = nn.LSTM(2, hidden_dim, num_layers=num_layers, batch_first=True) #lstm
#     self.lstm_1 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
#     self.fc_1 = nn.Linear(hidden_dim, hidden_dim)
#     self.fc = nn.Linear(hidden_dim, 1)
#     self.relu = nn.ReLU() 


#   def forward(self, input):
#     # LSTM
#     lstm, (hn, cn) = self.lstm(input)
#     lstm = nn.Dropout(0.2)(lstm)
#     # if DEBUG: print('lstm (batch, seq, feature):', lstm.shape)
#     lstm_1, (hn, cn) = self.lstm_1(lstm)
#     hn = nn.Dropout(0.2)(hn)
#     # hn = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)
#     hn = hn.view(-1, self.hidden_dim)
#     # if DEBUG: print('hn (batch, 1, feature):', hn.shape)

#     # candi = self.fc(candidate)

#     # out = torch.cat([hn], 1)
#     # if DEBUG: print('output:', out.shape)
#     # out = torch.cat([pool1, hn.reshape(64, 6, 1)], -1)
#     # if DEBUG: print('cat+sigmoid:', out.shape)
#     out = self.fc_1(hn).to(device) #first Dense
#     if DEBUG: print('fc_1:', out.shape)
#     out = self.relu(out).to(device) #relu
#     out = self.fc(out).to(device) #Final Output
#     # if DEBUG: print('fc:', out.shape)
#     # out = F.softmax(out, dim=1)
#     # if DEBUG: print('out:', out.shape)
#     return out


#### Bidirectional

In [ ]:
DEBUG = False

class LSTMTagger(nn.Module):

  def __init__(self, input_dim, embedding_dim, hidden_dim, num_classes, num_layers=1, seq_len=10):
    super(LSTMTagger, self).__init__()
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers

    # LSTM
    self.lstm = nn.LSTM(2, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True) #lstm
    self.lstm_1 = nn.LSTM(hidden_dim*2, hidden_dim, bidirectional=True, batch_first=True)
    self.fc_1 = nn.Linear(hidden_dim*2, hidden_dim)
    self.fc = nn.Linear(hidden_dim, N*M)
    self.relu = nn.ReLU() 


  def forward(self, input):
    # LSTM
    lstm, (hn, cn) = self.lstm(input)
    lstm = nn.Dropout(0.2)(lstm)
    if DEBUG: print('lstm (batch, seq, feature):', lstm.shape)
    lstm_1, (hn, cn) = self.lstm_1(lstm)
    hn = nn.Dropout(0.2)(hn)
    hn = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)
    # hn = hn.view(-1, self.hidden_dim)
    if DEBUG: print('hn (batch, 1, feature):', hn.shape)

    # candi = self.fc(candidate)

    # out = torch.cat([hn], 1)
    # if DEBUG: print('output:', out.shape)
    # out = torch.cat([pool1, hn.reshape(64, 6, 1)], -1)
    # if DEBUG: print('cat+sigmoid:', out.shape)
    out = self.fc_1(hn).to(device) #first Dense
    if DEBUG: print('fc_1:', out.shape)
    out = self.relu(out).to(device) #relu
    out = self.fc(out).to(device) #Final Output
    if DEBUG: print('fc:', out.shape)
    # out = F.softmax(out, dim=1)
    # if DEBUG: print('out:', out.shape)
    return out


# Train

In [ ]:
model = LSTMTagger(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, len(agent_to_ix)).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.005)
# scheduler = StepLR(optimizer, step_size=200, gamma=1)

for epoch in range(1000):  # again, normally you would NOT do 300 epochs, it is toy data
  model.train(True)
  for data in train_loader:
    inputs, targets = data
    outputs = model(inputs.type(torch.FloatTensor).to(device)).to(device)#.reshape(-1) #forward pass
    model.zero_grad()
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    if DEBUG: print(outputs.shape, targets.shape)
    loss = criterion(outputs, targets.to(device)) #obtain the loss function
    loss.backward() #calculates the loss of the loss function
    optimizer.step() #improve from loss, i.e backprop
    # scheduler.step()
    
  if (epoch + 1) % 50 == 0:
    print(loss.item())
    epoch_loss, epoch_acc = evaluate(model, val_loader)
    print("Epoch: %d, loss: %1.5f, acc: %1.5f" % (epoch+1, epoch_loss, epoch_acc))
    
    torch.save(model.state_dict(), f'e{epoch+1}.pkl')


LSTMTagger(
  (lstm): LSTM(2, 128, batch_first=True, bidirectional=True)
  (lstm_1): LSTM(256, 128, batch_first=True, bidirectional=True)
  (fc_1): Linear(in_features=256, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=2730, bias=True)
  (relu): ReLU()
)
3.9449541568756104
Epoch: 50, loss: 4.41312, acc: 0.03109
4.275480270385742
Epoch: 100, loss: 4.36084, acc: 0.07772
4.275128364562988
Epoch: 150, loss: 4.36033, acc: 0.07772
4.331498146057129
Epoch: 200, loss: 4.32505, acc: 0.07772
4.091116905212402
Epoch: 250, loss: 4.38641, acc: 0.03109
4.528120994567871
Epoch: 300, loss: 4.32002, acc: 0.07254
3.982429265975952
Epoch: 350, loss: 4.34361, acc: 0.03109
4.25377082824707
Epoch: 400, loss: 4.31273, acc: 0.06736
4.7101521492004395
Epoch: 450, loss: 4.32894, acc: 0.07772
4.201150894165039
Epoch: 500, loss: 4.30269, acc: 0.06218
3.7039785385131836
Epoch: 550, loss: 4.29915, acc: 0.08808
3.579759120941162
Epoch: 600, loss: 3.96142, acc: 0.08808
3.958127975463867
Epo

In [ ]:
model = LSTMTagger(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, len(agent_to_ix)).to(device)
model.load_state_dict(torch.load('e1000.pkl'))

for epoch in range(1000, 2000):  # again, normally you would NOT do 300 epochs, it is toy data
  model.train(True)
  for data in train_loader:
    inputs, targets = data
    outputs = model(inputs.type(torch.FloatTensor).to(device)).to(device)#.reshape(-1) #forward pass
    model.zero_grad()
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    if DEBUG: print(outputs.shape, targets.shape)
    loss = criterion(outputs, targets.to(device)) #obtain the loss function
    loss.backward() #calculates the loss of the loss function
    optimizer.step() #improve from loss, i.e backprop
    # scheduler.step()
    
  if (epoch + 1) % 50 == 0:
    print(loss.item())
    epoch_loss, epoch_acc = evaluate(model, val_loader)
    print("Epoch: %d, loss: %1.5f, acc: %1.5f" % (epoch+1, epoch_loss, epoch_acc))
    
    torch.save(model.state_dict(), f'e{epoch+1}.pkl')


2.201702117919922
Epoch: 1050, loss: 2.99298, acc: 0.21244
2.480130195617676
Epoch: 1100, loss: 2.94324, acc: 0.23316
2.4674177169799805
Epoch: 1150, loss: 2.93754, acc: 0.22280
2.7476470470428467
Epoch: 1200, loss: 2.98833, acc: 0.21244
3.0418026447296143
Epoch: 1250, loss: 2.96856, acc: 0.21762
2.4774961471557617
Epoch: 1300, loss: 2.96432, acc: 0.20207
2.523785352706909
Epoch: 1350, loss: 2.98537, acc: 0.23834
2.7396175861358643
Epoch: 1400, loss: 2.97561, acc: 0.24870
2.810131788253784
Epoch: 1450, loss: 2.97527, acc: 0.21762
2.5316250324249268
Epoch: 1500, loss: 2.99542, acc: 0.22280
2.237844228744507
Epoch: 1550, loss: 2.97577, acc: 0.21762
2.7548513412475586
Epoch: 1600, loss: 2.96763, acc: 0.20207
2.5008487701416016
Epoch: 1650, loss: 2.98172, acc: 0.20207
2.4413840770721436
Epoch: 1700, loss: 2.99800, acc: 0.19689
2.8648295402526855
Epoch: 1750, loss: 2.95066, acc: 0.22280
2.603304624557495
Epoch: 1800, loss: 2.99222, acc: 0.20725
2.5700929164886475
Epoch: 1850, loss: 2.98415,

In [ ]:
model = LSTMTagger(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, len(agent_to_ix)).to(device)
model.load_state_dict(torch.load('e2000.pkl'))
# evaluate(model, test_loader, True)
torch.save(model.state_dict(), 'e2000_bi_class_2_multi.pkl',_use_new_zipfile_serialization=False)

# Predicating

In [ ]:
cnx = sqlite3.connect('/content/drive/MyDrive/database_100+_train.db')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)
df_contacts_tmp = pd.read_sql_query("SELECT * FROM contacts JOIN walkers ON walkers.id = contacts.walker_id", cnx)
idx = df_contacts_tmp.groupby(['walker_id', 'time'])['distance'].transform(min) == df_contacts_tmp['distance']
df_contacts = df_contacts_tmp[idx]

DatabaseError: ignored

In [ ]:
id_to_pos, pos_to_id = utils.get_position(df_agents)
prob_agent_id, prob_dir, prob_move = utils.get_agent_next_prob(df_walkers, df_contacts, df_agents['id'].tolist())
prob_pos = utils.map_prob_to_pos(prob_agent_id, id_to_pos)

In [ ]:
cnx = sqlite3.connect('/content/drive/MyDrive/database_100-_test.db')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)
df_contacts_tmp = pd.read_sql_query("SELECT * FROM contacts JOIN walkers ON walkers.id = contacts.walker_id", cnx)
idx = df_contacts_tmp.groupby(['walker_id', 'time'])['distance'].transform(min) == df_contacts_tmp['distance']
df_contacts = df_contacts_tmp[idx]

In [ ]:
import utils
"""
Get counter of agent_id Key -> agent_id Values
"""
id_to_pos, pos_to_id = utils.get_position(df_agents)
prob_agent_id = utils.map_prob_to_agent_id(prob_pos, pos_to_id)

## Time Based

In [ ]:
walker_ids = set(df_contacts['walker_id'].tolist())
id_to_time = {}
id_to_agent = {}
start_time_to_id = collections.defaultdict(list)
for walker_id in walker_ids:
    candidates = df_contacts.loc[df_contacts['walker_id'] == walker_id].sort_values('time')
    min_time, max_time = candidates.iloc[0]['time'], candidates.iloc[-1]['time']
    id_to_time[walker_id] = (int(min_time), int(max_time))
    id_to_agent[walker_id] = (candidates.iloc[0]['agent_id'], candidates.iloc[-1]['agent_id'])
    start_time_to_id[int(min_time)].append(walker_id)

In [ ]:
walker_ids = set(df_contacts['walker_id'].tolist())
link_list = {}
for walker_id in walker_ids:
    last_time = id_to_time[walker_id][1]
    candidates = start_time_to_id[last_time + INTERVAL]

    if len(candidates) == 1:
        x0, y0 = id_to_agent[walker_id][1].split('EVEN')
        x1, y1 = id_to_agent[candidates[0]][0].split('EVEN')
        if abs(int(x0) - int(x1)) <= 1 and abs(int(y0)%30 - int(y1)%30) <= 1:
            link_list[walker_id] = candidates[0]
print(len(walker_ids))
print(link_list)

In [ ]:
# """
# Connect each linked path into a single path
# """
graph = nx.Graph([(i, j) for i, j in link_list.items() if i and j])
connected_components = sorted(nx.connected_components(graph), key=len, reverse=True)
print(len(connected_components))

tried = 0
for i, component in enumerate(connected_components):
    component = list(component)
    # print(component)
    new_id = component[0]
    time_set = set()
    for c in component[1:]:
        # rows = df_contacts.loc[df_contacts['walker_id'] == c]
            
        # tried += df_contacts.loc[df_contacts['walker_id'] == c].shape[0]
        df_contacts = df_contacts.replace({'walker_id': {c: new_id}})
        df_walks = df_walks.replace({'walker_id': {c: new_id}})

## Model Helper

In [ ]:
def prepare_data_test(walk, candidate, to_ix):
    path = []

    for i, row in walk.iterrows():
        path.append(to_ix[row['agent_id']])
    path.append(to_ix[candidate])
    return torch.Tensor([path]).type(torch.FloatTensor).to(device)

import math
walker_ids = set(df_contacts['walker_id'])
print(len(walker_ids))

In [ ]:
def compute_proba(df_contacts, walker_id1, walk, candidates, prob_agent_id, prob_dir, prob_move, clf=None):
    candidate_agent_probs = {}

    for _, candidate in candidates.iterrows():
        candidate_agent_probs[candidate['walker_id']] = model(prepare_data_test(walk, candidate['agent_id'], agent_to_ix))[0] #forward pass

    sum_prob = sum(candidate_agent_probs.values())
    for k, v in candidate_agent_probs.items():
      candidate_agent_probs[k] = candidate_agent_probs[k] / sum_prob
            
    return candidate_agent_probs

In [ ]:
walker_ids = set(df_contacts['walker_id'].tolist())
id_to_time = {}
id_to_agent = {}
start_time_to_id = collections.defaultdict(list)
for walker_id in walker_ids:
    candidates = df_contacts.loc[df_contacts['walker_id'] == walker_id].sort_values('time')
    min_time, max_time = candidates.iloc[0]['time'], candidates.iloc[-1]['time']
    id_to_time[walker_id] = (int(min_time), int(max_time))
    id_to_agent[walker_id] = (candidates.iloc[0]['agent_id'], candidates.iloc[-1]['agent_id'])
    start_time_to_id[int(min_time)].append(walker_id)

In [ ]:
from scipy.optimize import linear_sum_assignment
import math

link_list = {}
DEBUG = False
"""
Map agent id to its most possible next agent.
"""
model = LSTMTagger(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, len(agent_to_ix)).to(device)
model.load_state_dict(torch.load(f'e300.pkl', map_location=torch.device('cpu')))
walker_ids = set(df_contacts['walker_id'].tolist())
visited_walker_ids = set()
for i, walker_id1 in enumerate(walker_ids):
    if walker_id1 in visited_walker_ids:
        continue
    if i % 500 == 499:
        print(f'{i+1}...')
    walk = df_contacts.loc[df_contacts['walker_id'] == walker_id1].sort_values('time')
 
    time = walk.iloc[-1]['time']
    # get candidate points by time

    candidate_ids = start_time_to_id[time + INTERVAL]
    candidates = df_contacts.loc[df_contacts['walker_id'].isin(candidate_ids)]
 
    if candidates.shape[0] == 0:
        # print('no candidates')
        continue
 
    if candidates.shape[0] == 1:
        # print('1 candidates')
        link_list[walker_id] = candidates.iloc[-1]['walker_id']
        visited_walker_ids.add(walker_id)
        continue

    raw_batch_walker_ids = set(df_contacts.loc[df_contacts['time'] == time]['walker_id'])
    batch_walker_ids_prob = {}
    for walker_id2 in raw_batch_walker_ids:
        walk2 = df_contacts.loc[df_contacts['walker_id'] == walker_id2].sort_values('time')
        if int(walk2.iloc[-1]['time']) == time:
            batch_walker_ids_prob[walker_id2] = compute_proba(df_contacts, walker_id1, walk, candidates, prob_agent_id, prob_dir, prob_move)
 
    # print(batch_walker_ids_prob)
    walker_id_to_index, walker_index_to_id = {}, []
    candidate_id_to_index, candidate_index_to_id = {}, []
 
    for k, values in batch_walker_ids_prob.items():
        walker_id_to_index[k] = len(walker_id_to_index)
        walker_index_to_id.append(k)
        for c, v in values.items():
            if c not in candidate_id_to_index:
                candidate_id_to_index[c] = len(candidate_id_to_index)
                candidate_index_to_id.append(c)
    matrix = [[1e-32] * len(candidate_id_to_index) for _ in range(len(walker_id_to_index))]
    
    for k, values in batch_walker_ids_prob.items():
        for c, v in values.items():
            matrix[walker_id_to_index[k]][candidate_id_to_index[c]] = math.log(v)
                
    # print(matrix)
    row_ind, col_ind = linear_sum_assignment(matrix)
    # print(matrix)
    for row_idx, col_idx in zip(row_ind, col_ind):
        if matrix[row_idx][col_idx] == math.log(1e-32):
            continue
        link_list[walker_index_to_id[row_idx]] = candidate_index_to_id[col_idx]
        visited_walker_ids.add(walker_index_to_id[row_idx])

In [ ]:
"""
Connect each linked path into a single path
"""
graph = nx.Graph([(i, j) for i, j in link_list.items() if i and j])
connected_components = sorted(nx.connected_components(graph), key=len, reverse=True)
print(len(connected_components))
print(connected_components)

tried = 0
df_walks_connected = pd.DataFrame(df_walks)
for i, component in enumerate(connected_components):
    new_id = i
    time_set = set()
    for c in component:
        df_walks_connected = df_walks_connected.replace({'walker_id': {c: str(new_id) + 'res'}})
        

## Measurement

In [ ]:
"""
Precision: How much percentage of the grouped ids are really from one group
"""

DEBUG = True
walker_ids = set(df_walks_connected['walker_id'].tolist())
correct, total = 0, 0
if DEBUG: print(len(walker_ids))
    
for walker_id in walker_ids:
    id_list = df_walks_connected.loc[df_walks_connected['walker_id'] == walker_id].sort_values('walk_time')['real_id'].tolist()
    dic = collections.Counter(id_list)
    
    l = max(dic.values())
    key = [k for k,v in dic.items() if v == l]
    
    correct += l
    total += sum(dic.values())

# print(f'correct/tried: {correct}/{tried} {correct/tried*100}%') 
print(f'correct/total: {correct}/{total} {correct/total*100}%')

In [ ]:
"""
Accuracy: How much percentage of one group is correctly grouped together (only consider the largest sub-group of grouped ids)
"""

DEBUG = False
real_ids = set(df_walks_connected['real_id'].tolist())
correct, total = 0, 0
for real_id in real_ids:
    id_list = df_walks_connected.loc[df_walks_connected['real_id'] == real_id].sort_values('walk_time')['walker_id'].tolist()
    dic = collections.Counter(id_list)
    if DEBUG: print(id_list)
    
    l = max(dic.values())
    key = [k for k,v in dic.items() if v == l]
    
    correct += l
    total += sum([v for v in dic.values()])
#     df_walks_connected = df_walks_connected.replace({'walker_id': {key[0]: real_id}})
# print(f'base: {total - (150 / 10 - 1) * 10}/{total} {(total - (150 / 10 - 1) * 10)/total*100}')
# print(f'correct/tried: {correct}/{tried} {correct/tried*100}%') 
print(f'correct/total: {correct}/{total} {correct/total*100}%')

In [ ]:
DEBUG = False
real_ids = set(df_walks_connected['real_id'].tolist())
correct, total = 0, 0
visited = set()
for real_id in real_ids:
    id_list = df_walks_connected.loc[df_walks_connected['real_id'] == real_id].sort_values('walk_time')['walker_id'].tolist()
    dic = collections.Counter(id_list)
    if DEBUG: print(id_list)
    
    l = max(dic.values())
    p = [v for k,v in dic.items() if v == l][0] / sum(dic.values())
    if p > 0.6:
      correct += 1
    
#     df_walks_connected = df_walks_connected.replace({'walker_id': {key[0]: real_id}})
# print(f'base: {total - (150 / 10 - 1) * 10}/{total} {(total - (150 / 10 - 1) * 10)/total*100}')
# print(f'correct/tried: {correct}/{tried} {correct/tried*100}%') 
print(f'correct/total: {correct}/{len(real_ids)} {correct/len(real_ids)*100}%')